In [ ]:
!pip install mysql-connector boto3 cython PyHamcrest


In [1]:
endpoint = "example-aqs.ci1ykpa2ytax.us-east-2.rds.amazonaws.com" # Esta es la IP para la base de datos
lambdaRole = "arn:aws:iam::410253224468:role/cfst-1048-95df4023b0fe7844f9265c3f01cb-ALambdaRole-1396SFBQTLRG1" # Este es el rol de la función lambda
username = "example"
password = "example1."
dbname = "moviesdb"
tablename = "Movies_aqs"

In [2]:
#establece conexion RDS

import mysql.connector

mydb = mysql.connector.connect(
  host= endpoint,
  user= username,
  passwd= password
)

mycursor = mydb.cursor()



In [ ]:
#crea base de datos
mycursor = mydb.cursor()
mycursor.execute("CREATE DATABASE " + dbname)

In [3]:
#muestra las base de datos
mycursor.execute("SHOW DATABASES")

for x in mycursor:
    print(x)

(bytearray(b'example'),)
(bytearray(b'information_schema'),)
(bytearray(b'moviesdb'),)
(bytearray(b'mysql'),)
(bytearray(b'performance_schema'),)


In [6]:
#crear tabla 
mydb = mysql.connector.connect(
  host= endpoint,
  user= username,
  passwd= password,
  database= dbname
)
# 
mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE " + tablename + " (Year INT, Title VARCHAR(255), Actor VARCHAR(255), Rating INT, Runtime INT, Uploaded VARCHAR(255))")
mycursor.execute("ALTER TABLE " + tablename + " ADD COLUMN id INT AUTO_INCREMENT PRIMARY KEY")

In [ ]:
#base de datos dynamodb crea
import boto3
dynamodb = boto3.resource('dynamodb',  region_name='us-east-2')


table = dynamodb.create_table(
   TableName='Movies-aqs',
   KeySchema=[
       {
           'AttributeName': 'year',
           'KeyType': 'HASH'  #Partition key
       },
       {
           'AttributeName': 'title',
           'KeyType': 'RANGE'  #Sort key
       }
   ],
   AttributeDefinitions=[
       {
           'AttributeName': 'year',
           'AttributeType': 'N'
       },
       {
           'AttributeName': 'title',
           'AttributeType': 'S'
       },

   ],
   ProvisionedThroughput={
       'ReadCapacityUnits': 2,
       'WriteCapacityUnits': 2
   }
)

# Wait until the table exists.
table.meta.client.get_waiter('table_exists').wait(TableName='Movies')
print('Table is ready, please continue as instructed.')

In [11]:
#crea la funcion lambda
import boto3
client = boto3.client('lambda',  region_name='us-east-2')
lambdarole = "arn:aws:iam::410728552766:role/lambda_permiso"

response = client.create_function(
    FunctionName='carga_archivosdynamo_aqs_v2',
    Runtime='nodejs12.x',
    Role= lambdarole,
    Handler='index.handler',
    Code={
        'ZipFile': open('/home/ec2-user/SageMaker/aquevedo/ddbStream.zip', 'rb').read()
    },
    Description='Extracts from DynamoDB Stream and adds to Relational DB',
    Timeout=5,
    Environment={
        'Variables': {
            'endPoint': endpoint
        }
    }
)

In [15]:
#carga datos a dynamodb y automaticamente carga a mysql
from __future__ import print_function # Python 2/3 compatibility
import boto3
import json
import decimal
import sys
import random
dynamodb = boto3.resource('dynamodb',  region_name='us-east-2')
table = dynamodb.Table('Movies-aqs')

choices = ['yes', 'no']
i = 0
with open("/home/ec2-user/SageMaker/aquevedo/moviedata.json") as json_file:
    with table.batch_writer() as batch:
        movies = json.load(json_file, parse_float = decimal.Decimal)
        for movie in movies:
            i = i + 1
            if i == 101:
                break
            year = int(movie['year'])
            title = movie['title']
            star = movie['actors'][0]
            rating = movie['rating']
            running_time = movie['running_time_secs']
            uploaded = random.choice(choices)

            print("Adding movie:", year, title, star, rating, running_time, uploaded)

            batch.put_item(
               Item={
                   'year': year,
                   'title': title,
                   'actor': star,
                   'rating': rating,
                   'running_time': running_time,
                   'uploaded' : uploaded
                }
            )


Adding movie: 2013 Rush Daniel Bruhl 8.3 7380 no
Adding movie: 2013 Prisoners Hugh Jackman 8.2 9180 no
Adding movie: 2013 This Is the End James Franco 7.2 6420 yes


In [17]:
import pandas as pd

mydb.commit()
mycursor = mydb.cursor()
mycursor.execute("SELECT * FROM moviesdb." + tablename +";")

myresult = mycursor.fetchall()

item_list = []
for i in myresult:
    item = {'id':i[6], 
            'title':i[1], 
            'actor' :i[2], 
            'rating' :i[3], 
            'running_time' :i[4], 
            'uploaded' :i[5],
            'year' :i[0]}
    item_list.append(item)
df = pd.DataFrame(data=item_list,columns=['id','year','title','actor','running_time','rating','uploaded'])
df.head(30)

,id,year,title,actor,running_time,rating,uploaded
0,1,2013,Rush,Daniel Bruhl,7380,8,no
1,2,2013,Prisoners,Hugh Jackman,9180,8,no
2,3,2013,This Is the End,James Franco,6420,7,yes


In [14]:
mycursor = mydb.cursor()
mycursor.execute("truncate table moviesdb." + tablename)